In [ ]:
import pandas as pd
import numpy as np
import concurrent.futures
import scipy.signal as signal
from scipy import interpolate

In [ ]:
def noise(x, amp):
    return x + (np.random.randn(len(x)) * amp)

def ismonotonic(x):
    max_peaks = signal.argrelextrema(x, np.greater)[0]
    min_peaks = signal.argrelextrema(x, np.less)[0]
    all_num = len(max_peaks) + len(min_peaks)
    if all_num > 0:
        return False
    else:
        return True

def findpeaks(x):
    return signal.argrelextrema(x, np.greater)[0]

def isimf(x):
    N = np.size(x)
    pass_zero = np.sum(x[0:N - 2] * x[1:N - 1] < 0)
    peaks_num = np.size(findpeaks(x)) + np.size(findpeaks(-x))
    if abs(pass_zero - peaks_num) > 1:
        return False
    else:
        return True

def mirror(x):
    i = np.arange(len(x) - 2, 0, -1)
    reverse_x = x[i]
    reverse_x = list(reverse_x)
    x = list(x)
    return np.array(reverse_x + x + reverse_x)

def getspline(x):
    N = np.size(x)
    peaks = findpeaks(x)
    if len(peaks) <= 3:
        if len(peaks) < 2:
            peaks = np.concatenate(([0], peaks))
            peaks = np.concatenate((peaks, [N - 1]))
        t = interpolate.splrep(peaks, y=x[peaks], w=None, xb=None, xe=None, k=len(peaks) - 1)
        return interpolate.splev(np.arange(N), t)
    t = interpolate.splrep(peaks, y=x[peaks])
    return interpolate.splev(np.arange(N), t)

def emd(x):
    imf = []
    while not ismonotonic(x):
        x1 = x
        sd = np.inf
        while sd > 0.2 or (not isimf(x1)):
            x1 = mirror(x1)
            s1 = getspline(x1)
            s2 = -getspline(-1 * x1)
            x2 = x1 - (s1 + s2) / 2
            x1 = x1[len(x) - 2:-len(x) + 2]
            x2 = x2[len(x) - 2:-len(x) + 2]
            sd = np.sum((x1 - x2) ** 2) / np.sum(x1 ** 2)
            x1 = x2
        imf.append(x1)
        x = x - x1
    imf.append(x)
    return imf

def eemd(x, amp, times, n_modes):
    x = x.reshape(-1)
    imf, n = 0, 0
    for _ in range(times):
        modes = emd(noise(x, amp))
        if len(modes) == n_modes:
            imf += np.array(modes)
            n += 1
    print(f'Number of EMDs: {n}')
    for i in range(len(imf)):
        imf[i] = imf[i] / n
    return imf

def Pool(x):
    return eemd(x=x, amp=400, times=10, n_modes=11)

In [ ]:
data = pd.read_csv('bdi.csv').fillna(method='ffill')['bdi'].values
index = [data[:i] for i in range(-1523, 0)]
index.append(data[:])

In [ ]:
with concurrent.futures.ProcessPoolExecutor() as executor:
    results = executor.map(Pool, index)
    for index, result in enumerate(results):
        np.save(f'EEMDs/EEMD{index}', result)